In [1]:
from sklearn.datasets import make_blobs
import pandas as pdm
import numpy as np
import math
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import os
import yaml
import pandas as pd
import ast


In [2]:
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute

In [3]:
# MEMO: top100 games
with open("C:\\Users\weejw\Documents\steamRecommendation\leanring\id_list.txt","r") as file:
    id_list = file.readlines()

app_ids = ast.literal_eval(id_list[0])

# date get from database

In [4]:
with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)
try:
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])

    #sql = "SELECT * FROM game_info"
    # MEMO: TOP 100 games
    game_info_sql = f"SELECT * FROM game_info where steam_appid in {str(tuple(app_ids))}"
    review_sql = f"SELECT * FROM review where steam_appid in {str(tuple(app_ids))}"
    
    game_info_rows = db_function.execute_select(game_info_sql)
    review_rows = db_function.execute_select(review_sql)
    
except:
    raise
    


In [5]:
len(game_info_rows), len(review_rows)

(95, 5649)

# game info 정제

In [6]:
game_info_cols = ['id', 'steam_appid', 'name', 'required_age', 'is_free', 'detailed_description', 'about_the_game',
       'short_description', 'developers', 'publishers', 'initial_price', 'final_price', 'discount_percent',
       'platforms', 'release_date', 'supported_languages', 'categories', 'genres', 'recommendations', 'categories_id',
       'genres_id']

In [7]:
game_info = pd.DataFrame(game_info_rows, columns=game_info_cols)

In [8]:
need_cols=['steam_appid','name','recommendations', 'categories_id', 'genres_id', 'mac', 'linux','windows', 'initial_price', 'final_price', 'discount_percent', 'supported_languages']

In [9]:
game_info.columns

Index(['id', 'steam_appid', 'name', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'developers', 'publishers', 'initial_price', 'final_price',
       'discount_percent', 'platforms', 'release_date', 'supported_languages',
       'categories', 'genres', 'recommendations', 'categories_id',
       'genres_id'],
      dtype='object')

In [10]:
game_info.fillna(0, inplace=True)

In [11]:
game_info['windows']=game_info['platforms'].apply(lambda x:1 if 'windows' in x else 0)
game_info['linux']=game_info['platforms'].apply(lambda x:1 if 'linux' in x else 0)
game_info['mac']=game_info['platforms'].apply(lambda x:1 if 'mac' in x else 0)

In [12]:
game_info['initial_price'] = game_info['initial_price'].apply(lambda x:int(x*0.01))
game_info['final_price'] = game_info['final_price'].apply(lambda x:int(x*0.01))
game_info['recommendations'] = game_info['recommendations'].apply(lambda x:int(x))

In [13]:
game_info = game_info[need_cols]

# review 정제

In [14]:
review_cols = ['id', 'steam_appid', 'steamid', 'num_games_owned', 'playtime_last_two_weeks', 'playtime_at_review',
                             'last_played', 'votes_up', 'votes_funny', 'received_for_free', 'written_during_early_access',
                             'steam_purchase', 'num_reviews', 'playtime_forever', 'review_type', 'review']

In [15]:
reviews = pd.DataFrame(review_rows, columns=review_cols)

In [16]:
reviews

,id,steam_appid,steamid,num_games_owned,playtime_last_two_weeks,playtime_at_review,last_played,votes_up,votes_funny,received_for_free,written_during_early_access,steam_purchase,num_reviews,playtime_forever,review_type,review
0,16166,10,76561197967907928,125,0,6731,2018-11-25 20:56:35,2,0,0,0,1,27,8034,positive,말이 필요없음
1,16184,10,76561197973786146,86,0,10,2021-09-04 15:31:22,0,0,0,0,1,7,16,positive,This is where Valve has created the seed of ST...
2,16186,10,76561197974087280,40,0,20265,2014-09-08 23:51:44,0,0,0,0,1,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,16222,10,76561197985593224,94,0,9,2020-06-29 09:23:47,0,0,0,0,1,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,18713,10,76561197995131409,169,0,35,2020-02-14 20:56:10,0,0,0,0,1,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,16912,1468810,76561198031355345,34,1910,8610,2021-12-12 14:57:31,29,1,0,1,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,16914,1468810,76561198036523483,77,0,3510,2021-09-15 13:59:24,1,0,0,1,1,9,4466,positive,GOOD
5646,19498,1468810,76561198095212841,399,0,970,2021-01-31 04:26:58,6,4,0,1,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,19497,1468810,76561198203238402,183,0,29,2021-03-30 18:49:41,7,0,0,1,1,242,29,negative,add korean or english language so i can actua...


In [17]:
need_cols=['steam_appid', 'num_games_owned', 'playtime_last_two_weeks', 'playtime_at_review', 'votes_up', 'received_for_free', 'written_during_early_access', 'num_reviews', 
          'playtime_forever', 'review_type', 'review']

In [18]:
reviews = reviews[need_cols]

In [19]:
reviews

,steam_appid,num_games_owned,playtime_last_two_weeks,playtime_at_review,votes_up,received_for_free,written_during_early_access,num_reviews,playtime_forever,review_type,review
0,10,125,0,6731,2,0,0,27,8034,positive,말이 필요없음
1,10,86,0,10,0,0,0,7,16,positive,This is where Valve has created the seed of ST...
2,10,40,0,20265,0,0,0,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,10,94,0,9,0,0,0,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,10,169,0,35,0,0,0,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...
5644,1468810,34,1910,8610,29,0,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,1468810,77,0,3510,1,0,1,9,4466,positive,GOOD
5646,1468810,399,0,970,6,0,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,1468810,183,0,29,7,0,1,242,29,negative,add korean or english language so i can actua...


# Merga

In [20]:
all_data = pd.merge(game_info, reviews)

In [21]:
all_data

,steam_appid,name,recommendations,categories_id,genres_id,mac,linux,windows,initial_price,final_price,...,num_games_owned,playtime_last_two_weeks,playtime_at_review,votes_up,received_for_free,written_during_early_access,num_reviews,playtime_forever,review_type,review
0,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,125,0,6731,2,0,0,27,8034,positive,말이 필요없음
1,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,86,0,10,0,0,0,7,16,positive,This is where Valve has created the seed of ST...
2,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,40,0,20265,0,0,0,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,94,0,9,0,0,0,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,169,0,35,0,0,0,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,34,1910,8610,29,0,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,77,0,3510,1,0,1,9,4466,positive,GOOD
5646,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,399,0,970,6,0,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,183,0,29,7,0,1,242,29,negative,add korean or english language so i can actua...


In [22]:
all_data.columns

Index(['steam_appid', 'name', 'recommendations', 'categories_id', 'genres_id',
       'mac', 'linux', 'windows', 'initial_price', 'final_price',
       'discount_percent', 'supported_languages', 'num_games_owned',
       'playtime_last_two_weeks', 'playtime_at_review', 'votes_up',
       'received_for_free', 'written_during_early_access', 'num_reviews',
       'playtime_forever', 'review_type', 'review'],
      dtype='object')

# 리뷰 수와 리뷰 찬반율과 추천 수 세가지를 이용해서?

In [23]:
rate_table = pd.pivot_table(all_data[['steam_appid','review_type','review']], index='steam_appid', columns=['review_type'], aggfunc = ['count'])

In [24]:
rate_table

count         
              review         
review_type negative positive
steam_appid                  
10              25.0    100.0
30               7.0     16.0
40               6.0      9.0
50              13.0    100.0
60               8.0     11.0
...              ...      ...
1097150         76.0     48.0
1172470          NaN      1.0
1203220         16.0     19.0
1240440          7.0     16.0
1468810          3.0      3.0

[93 rows x 2 columns]

In [25]:
rate_table = rate_table.droplevel(level=[0,1], axis=1)

In [26]:
rate_table['positive_rate'] = rate_table['positive'] / (rate_table['positive']+rate_table['negative'])

In [27]:
rate_table = rate_table.sort_values(by='positive_rate').fillna(0)

In [28]:
rate_table

review_type,negative,positive,positive_rate
steam_appid,,,
1089350,100.0,1.0,0.009901
252950,98.0,1.0,0.010101
96000,18.0,1.0,0.052632
440,41.0,3.0,0.068182
220,71.0,7.0,0.089744
...,...,...,...
363970,0.0,4.0,0.000000
386360,2.0,0.0,0.000000
417910,2.0,0.0,0.000000


In [31]:
game_info[game_info.steam_appid.apply(lambda x:x in rate_table.index[:10])]

,steam_appid,name,recommendations,categories_id,genres_id,mac,linux,windows,initial_price,final_price,discount_percent,supported_languages
5,70,Half-Life,53811,"2,1,49,36,23,8,41,42,44",1,1,1,1,10500,10500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 스페인어 스페인, 중국어 간체, 중국어 번..."
8,220,Half-Life 2,100493,"2,22,29,13,18,23,16,41,42,44",1,1,1,1,10500,10500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 한국어, 스페인어 스페인, 러시아어, 중국..."
9,240,Counter-Strike: Source,85389,"1,27,22,23,8,15,16",1,1,1,1,10500,10500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 일본어, 한국어, 스페인어 스페인, 러시아..."
14,440,Team Fortress 2,12786,"1,27,22,29,13,30,35,18,8,15,17,14,41,42","1,37",1,1,1,0,0,0.0,"영어, 덴마크어, 네덜란드어, 핀란드어, 프랑스어, 독일어, 이탈리아어, 일본어, ..."
17,620,Portal 2,219126,"2,1,9,38,39,24,22,28,29,13,51,30,23,15,17,14,4...","1,25",1,1,1,10500,10500,0.0,"영어, 프랑스어, 독일어, 스페인어 스페인, 체코어, 덴마크어, 네덜란드어, 핀란..."
24,96000,The Tiny Bang Story,3192,"2,29","25,4,23",1,1,1,5500,1370,75.0,"영어, 러시아어, 프랑스어, 이탈리아어, 독일어, 스페인어 스페인, 불가리아어, ..."
29,218620,PAYDAY 2,351585,"2,1,9,38,22,28,29,35,23,41,42,43","1,3",0,1,1,10500,10500,0.0,"영어, 독일어, 프랑스어, 이탈리아어, 스페인어 스페인, 네덜란드어, 러시아어, ..."
40,252950,Rocket League®,405107,"2,1,49,36,37,9,38,39,24,27,22,28,29,30,23,15,4...","1,23,9,18",0,0,1,0,0,0.0,"영어, 프랑스어, 이탈리아어, 독일어, 스페인어 스페인, 네덜란드어, 포르투갈어,..."
41,255710,Cities: Skylines,132425,"2,22,29,30,18,23,17,42","28,2",1,1,1,32000,32000,0.0,"영어, 프랑스어, 독일어, 스페인어 스페인, 폴란드어, 포르투갈어 브라질, 러시..."
88,1089350,NBA 2K20,45760,"2,1,49,36,37,9,38,39,24,22,28,35,23,41,42,43,44","28,18",0,0,1,65000,65000,0.0,"영어, 프랑스어, 이탈리아어, 독일어, 스페인어 스페인, 일본어, 한국어, 중국어..."
